<a href="https://colab.research.google.com/github/skywalker00001/Conterfactual-Reasoning-Project/blob/main/data_cleaning_6_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# two stages

# how to find out the different or changed blocks in sentence b compared to the sentence a (base)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
root = 'drive/MyDrive/LM/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#!pip install sentencepiece
#!pip install transformers -q

In [3]:
# Importing stock libraries
import numpy as np
import pandas as pd
import difflib
import nltk
import copy
import regex as re
import sys
import torch
sys.path.append('/content/drive/MyDrive/LM/')
from global_param import MyConfig
from tqdm import tqdm
nltk.download("punkt")
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenizer = TreebankWordDetokenizer()

# Importing the T5 modules from huggingface/transformers
# from transformers import T5Tokenizer, T5ForConditionalGeneration

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
myconfig = MyConfig()

In [5]:
#PRETRAINED_MODEL_NAME = myconfig.PRETRAINED_MODEL_NAME
# tokenzier for encoding the text
#t5_tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [6]:
# nltk.word_tokenize() as tokenizer
def words_origin2backbone(origin, edited):
    #origin_list = origin.split(" ")
    origin_list = nltk.word_tokenize(origin)
    #print(origin_list)
    #edited_list = edited.split(" ")
    edited_list = nltk.word_tokenize(edited)
    s = difflib.SequenceMatcher(None, origin_list, edited_list)
    cgs, answers = [], []
    counter = 0
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        if (tag == 'equal'):
            cgs.extend(edited_list[j1: j2])
            #cgs.append(f'{i1}-{i2}:\"{(" ").join(edited_list[j1:j2])}\".')
            #cgs.append(f'{i1}<extra_id_0>{i2}<extra_id_1>{(" ").join(edited_list[j1:j2])}<extra_id_2>')
        elif (tag == 'insert' or tag == 'replace'): 
            cgs.append(f'<extra_id_{counter}>')
            answers.append(f'<extra_id_{counter}>')
            counter += 1
            answers.append(detokenizer.detokenize(edited_list[j1: j2]).replace(' , ',',').replace(' .','.') \
                           .replace(' !','!').replace(' ?','?').replace(' : ',': ').replace(' \'', '\'')  )
    #print(cgs)
    conclusion = detokenizer.detokenize(cgs)
    conclusion = conclusion.replace(' , ',',').replace(' .','.').replace(' !','!').replace(' ?','?').replace(' : ',': ').replace(' \'', '\'')   
    answers = ("").join(answers)
    return conclusion, answers

# 1. Preparation

In [7]:
#training df
small_path = root + '/TimeTravel/train_supervised_small.json'
small_df = pd.read_json(small_path, lines=True)
small_df.head()
print(len(small_df))

16752


In [8]:
# Data cleaning, remove the examples with the same original_ending and edited_ending

# comb_small_text=[]

# for i in range(len(small_df)):

#   comb_small_text.append(small_df.loc[i, 'edited_ending'][0] +" "+ small_df.loc[i, 'edited_ending'][1] +" "+ \
#                 small_df.loc[i, 'edited_ending'][2])
# small_df.insert(6, "ground_truth", comb_small_text)
# small_cleaned_df = small_df[small_df.ground_truth != small_df.original_ending]
# small_cleaned_df = small_cleaned_df.reset_index() 
# print(len(small_cleaned_df))

In [9]:
# # Data cleaning, remove the examples with the same original_ending and edited_ending

# comb_small_text=[]

# for i in range(len(small_df)):
#     edited = ""
#     for j in range(len(small_df.loc[i, 'edited_ending'])):
#         edited += small_df.loc[i, 'edited_ending'][j]
#         if (edited[-1] != "."):
#             edited += "."
#         if (j != len(small_df.loc[i, 'edited_ending']) - 1):
#             edited += " "
        
#     comb_small_text.append(edited)
#     # comb_small_text.append(small_df.loc[i, 'edited_ending'][0] +" "+ small_df.loc[i, 'edited_ending'][1] +" "+ \
#     #               small_df.loc[i, 'edited_ending'][2])
# small_df.insert(6, "ground_truth", comb_small_text)
# small_cleaned_df = small_df[small_df.ground_truth != small_df.original_ending]
# small_cleaned_df = small_cleaned_df.reset_index() 
# print(len(small_cleaned_df))

In [10]:
# input1: source, output1: target, 
premise, initial, counterfactual, original_ending, edited_ending, input1, output1, output2 = [], [], [], [], [], [], [], []
bar = tqdm(range(len(small_df)))
for i in bar:

    premise.append(small_df.loc[i, 'premise'])
    initial.append(small_df.loc[i, 'initial'])
    counterfactual.append(small_df.loc[i, 'counterfactual'])
    original_ending.append(small_df.loc[i, 'original_ending'])
    edited = ""
    for j in range(len(small_df.loc[i, 'edited_ending'])):
        text = small_df.loc[i, 'edited_ending'][j]
        if (text[-1] == ","):
            edited += text[0:-1] + "."
        else:
            edited += text
        #edited += small_df.loc[i, 'edited_ending'][j]
        #if (edited[-1] != "." and edited[-1] != "!" and edited[-1] != "?"):
        #    edited += "."
        if (edited[-1] != "." and edited[-1] != "!" and edited[-1] != "?"):
            edited += "."
        if (j != len(small_df.loc[i, 'edited_ending']) - 1):
            edited += " "
    edited_ending.append(edited)

    input1.append("premise: " + small_df.loc[i, 'premise']  + \
                  " initial: " + small_df.loc[i, 'initial']  + \
                  " original_ending: " + small_df.loc[i, 'original_ending'] + \
                  " counterfactual: " + small_df.loc[i, 'counterfactual'] 
                  )
    
    conclusion, answers = words_origin2backbone(small_df.loc[i, 'original_ending'], edited)
    output1.append(conclusion)
    output2.append(answers)

100%|██████████| 16752/16752 [00:34<00:00, 482.82it/s]


In [11]:
#premise, initial, counterfactual, original_ending, edited_ending, input1, output1, output2
uncleaned_train_df = pd.DataFrame({'premise': premise, 'initial': initial, 'counterfactual': counterfactual, 'original_ending': original_ending, \
                         'edited_ending': edited_ending, 'input1': input1, 'output1': output1, 'output2': output2}) 
uncleaned_train_df.head()
print(len(uncleaned_train_df))

16752


In [12]:
# Data cleaning, remove the examples with the same original_ending and edited_ending

#train_df = uncleaned_train_df[uncleaned_train_df.output2 != uncleaned_train_df.original_ending]
train_df = uncleaned_train_df[uncleaned_train_df.output2 != ""]
train_df = train_df.reset_index(drop=True) 
print(len(train_df))

15702


In [13]:
#train_df.to_excel(root + 'TimeTravel/' + 'output' + model_version + '.xlsx')
train_df.to_excel(root + 'TimeTravel/' + 'cleaned_small_2.0.xlsx', )